# Get Raw Price Data

In [8]:
import subprocess

In [16]:
import subprocess
proc = subprocess.Popen(["curl", url_template], stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
print("program output: {0}".format(out))

program output: b''


In [103]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from utils import COOKIES, HEADERS, REQUEST_DATA, COLUMNS
import threading
import numpy as np

In [5]:
def get_prices_date(date):

    data = REQUEST_DATA + [
      ('fecha', date),
      ('fechaDesde', date),
      ('fechaHasta', date),
    ]

    return requests.post('http://www.lasalina.es/Aplicaciones/GestorInter.jsp', headers=HEADERS, cookies=COOKIES, data=data)

In [23]:
def get_prices_df(prices):
    
    dict_data = {col:[] for col in COLUMNS}        
    [[dict_data[col].append(price.find_all("td")[idx].text) for idx, col in enumerate(COLUMNS)] for price in prices]
    
    return pd.DataFrame.from_dict(data = dict_data)[COLUMNS]

In [92]:
def write_prices_single_date(date):
    
    soup = BeautifulSoup(get_prices_date(date).text, 'html.parser')
    prices = soup.find_all("table")[1].find_all("tr")[1:]
    get_prices_df(prices).to_csv("{0}_{1}.{2}".format("raw_data/data", date, 'csv'),index = False)

In [93]:
def write_prices_list_dates(list_dates):
    [write_prices_single_date(date) for date in list_dates]

In [105]:
def get_dates():
    
    dt = datetime.datetime(2006, 1, 1)
    end_date = datetime.datetime(2018, 8, 13)
    step = datetime.timedelta(weeks = 1)
    result = []
    while dt < end_date:
        result.append(dt.strftime('%d-%m-%Y'))
        dt += step
    return result

In [106]:
dates = get_dates()
num_threads = 4
dates_array_splits = np.array_split(dates, num_threads)

jobs = []
for date_array in dates_array_splits:
    thread = threading.Thread(target=write_prices_list_dates(list(date_array)))
    jobs.append(thread)
    
for j in jobs:
    j.start()

for j in jobs:
    j.join()